In [1]:
import rasterio as rio
import numpy as np
import pandas as pd
import datetime
import os

In [ ]:
# read in centerline velocity data (grab mid_date and satellite)

# calculate 1D gradient in velocity along centerline

# plot centerline velocity vs. distance and centerline velocity gradient vs. distance

# pick velocity gradient threshold and spit out the distance from terminus

# plot that distance as a line on the map

In [ ]:
# replot surge front distance from terminus vs. date figure with vmap